# Model 3: user-based/collaborative filtering (ALS)
- Recommend games to users based on similar users’ preferences.
- input: purchased_games, history
- This code run successfully on Google Colab

In [1]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

spark_version = "3.5.5"
!wget -q https://dlcdn.apache.org/spark/spark-{spark_version}/spark-{spark_version}-bin-hadoop3.tgz

!tar xf spark-{spark_version}-bin-hadoop3.tgz

!pip install findspark

!pip install pyspark==3.5.5

In [2]:
import findspark, os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.5-bin-hadoop3"

findspark.init()

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SteamGameRecommender") \
    .config("spark.driver.memory", "8g") \
    .getOrCreate()

In [4]:
from google.colab import drive

drive.mount('/content/drive')
mount_point = "/content/drive/MyDrive/colab_mount/"

Mounted at /content/drive


In [5]:
purchased_games = spark.read.csv(f'{mount_point}purchased_games_demo.csv', header=True, inferSchema=True)

history = spark.read.csv(f'{mount_point}history.csv', header=True, inferSchema=True)

games = spark.read.csv(f'{mount_point}games.csv', header=True, inferSchema=True)

In [6]:
purchased_games.show()

+-----------------+--------------------+
|         playerid|             library|
+-----------------+--------------------+
|76561198060698936|[60, 1670, 3830, ...|
|76561198287452552|[10, 80, 100, 240...|
|76561198040436563|[10, 80, 100, 300...|
|76561198042412488|[300, 240, 220, 3...|
|76561198119605821|[47870, 108600, 5...|
|76561198049686270|                  []|
|76561198155814250|                  []|
|76561198083492916|[1300, 1309, 1313...|
|76561198150634683|[10, 30, 40, 60, ...|
|76561198836367256|[2600, 15100, 500...|
|76561198131958442|[10, 80, 100, 70,...|
|76561198106910534|                  []|
|76561198122070915|[50, 70, 130, 120...|
|76561198740883087|                  []|
|76561198016756834|[10, 80, 100, 20,...|
|76561198072740562|[1510, 4000, 9453...|
|76561198144540723|                  []|
|76561198096557299|                  []|
|76561198884939938|[4000, 304930, 39...|
|76561198220441373|[2100, 2130, 3830...|
+-----------------+--------------------+
only showing top

In [7]:
history.show()

+-----------------+-------------+-------------------+
|         playerid|achievementid|      date_acquired|
+-----------------+-------------+-------------------+
|76561198220441373| 403640_ACH_1|2019-12-18 15:33:43|
|76561198220441373| 403640_ACH_2|2019-12-18 23:49:51|
|76561198220441373| 403640_ACH_3|2019-12-19 23:05:07|
|76561198220441373| 403640_ACH_4|2019-12-24 05:50:49|
|76561198220441373| 403640_ACH_5|2023-04-19 22:39:36|
|76561198220441373| 403640_ACH_6|2023-04-30 02:48:24|
|76561198220441373| 403640_ACH_7|2023-04-25 23:59:38|
|76561198220441373| 403640_ACH_8|2023-05-07 16:03:48|
|76561198220441373| 403640_ACH_9|2022-05-15 18:07:07|
|76561198220441373|403640_ACH_10|2023-01-24 18:40:15|
|76561198220441373|403640_ACH_13|2023-04-12 22:58:19|
|76561198220441373|403640_ACH_21|2023-04-24 22:51:07|
|76561198220441373|403640_ACH_23|2023-04-30 02:57:09|
|76561198220441373|403640_ACH_24|2023-05-06 23:43:09|
|76561198220441373|403640_ACH_25|2023-05-06 22:15:59|
|76561198220441373|403640_AC

In [8]:
from pyspark.sql.functions import split, col, count, regexp_replace, explode, coalesce, lit

In [9]:
# Extract gameid from achievementid
history_extracted = history.withColumn("gameid", split(col("achievementid"), "_")[0])

In [10]:
history_extracted.show()

+-----------------+-------------+-------------------+------+
|         playerid|achievementid|      date_acquired|gameid|
+-----------------+-------------+-------------------+------+
|76561198220441373| 403640_ACH_1|2019-12-18 15:33:43|403640|
|76561198220441373| 403640_ACH_2|2019-12-18 23:49:51|403640|
|76561198220441373| 403640_ACH_3|2019-12-19 23:05:07|403640|
|76561198220441373| 403640_ACH_4|2019-12-24 05:50:49|403640|
|76561198220441373| 403640_ACH_5|2023-04-19 22:39:36|403640|
|76561198220441373| 403640_ACH_6|2023-04-30 02:48:24|403640|
|76561198220441373| 403640_ACH_7|2023-04-25 23:59:38|403640|
|76561198220441373| 403640_ACH_8|2023-05-07 16:03:48|403640|
|76561198220441373| 403640_ACH_9|2022-05-15 18:07:07|403640|
|76561198220441373|403640_ACH_10|2023-01-24 18:40:15|403640|
|76561198220441373|403640_ACH_13|2023-04-12 22:58:19|403640|
|76561198220441373|403640_ACH_21|2023-04-24 22:51:07|403640|
|76561198220441373|403640_ACH_23|2023-04-30 02:57:09|403640|
|76561198220441373|40364

In [11]:
# Group by playerid and gameid, count achievements
achievement_counts = history_extracted.groupBy("playerid", "gameid") \
                                   .agg(count("achievementid").alias("achievement_count"))

In [12]:
achievement_counts.show()

+-----------------+-------+-----------------+
|         playerid| gameid|achievement_count|
+-----------------+-------+-----------------+
|76561198220441373| 526870|               11|
|76561197997318138| 286320|               10|
|76561197997318138| 316610|                5|
|76561197997318138| 258520|               14|
|76561197997318138| 359310|               38|
|76561198046388487| 219640|                1|
|76561198056486056| 291010|                2|
|76561199172501672|2120900|                1|
|76561198008683936| 211400|               13|
|76561198008683936| 257420|               54|
|76561198212587903| 360430|               85|
|76561198099373691| 238320|                9|
|76561197987596512| 314020|               13|
|76561197987596512| 575950|             2995|
|76561198369428194| 304530|                9|
|76561198093121593|    550|                4|
|76561198093121593| 275850|               15|
|76561198267894519| 434460|                2|
|76561198071709714|1100410|       

In [15]:
# print playerid with max achievement_count
achievement_counts.orderBy(col("achievement_count").desc()).show(100)

+-----------------+------+-----------------+
|         playerid|gameid|achievement_count|
+-----------------+------+-----------------+
|76561198099809175|664290|            10979|
|76561198298591193|664290|            10979|
|76561197987340453|664290|            10979|
|76561198071675053|664290|            10979|
|76561198324276106|664290|            10979|
|76561198298591193|573060|             9821|
|76561198324276106|573060|             9821|
|76561198071675053|573060|             9821|
|76561198132280885|664290|             7985|
|76561198307860947|664290|             5989|
|76561197987340453|640310|             5394|
|76561198298591193|640310|             5394|
|76561198099241446|640310|             5394|
|76561198398342040|640310|             5394|
|76561198314830742|640310|             5394|
|76561198071675053|640310|             5394|
|76561198324276106|640310|             5394|
|76561198206518029|640310|             5394|
|76561198168918509|640310|             5294|
|765611983

In [16]:
# Step 1: Get list of playerids who have achievements
valid_players_df = achievement_counts.select("playerid").distinct()

In [17]:
valid_players_df.show()

+-----------------+
|         playerid|
+-----------------+
|76561198389473114|
|76561198024318505|
|76561199643317571|
|76561198138419099|
|76561198139094658|
|76561198012584047|
|76561198139850525|
|76561197998500968|
|76561198366833428|
|76561199092807644|
|76561198298591193|
|76561198045587928|
|76561198453328904|
|76561198868227222|
|76561198204258625|
|76561198844100946|
|76561198405685043|
|76561198189044965|
|76561198367651601|
|76561197980310845|
+-----------------+
only showing top 20 rows



In [18]:
# Step 1: remove brackets, spaces, and split into array of strings
purchased_games = purchased_games.withColumn(
    "library",
    split(regexp_replace("library", r"[\[\]\s]", ""), ",")
)

In [19]:
# Explode the array into individual rows
user_game = purchased_games.select(col("playerid"), explode(col("library")).alias("gameid"))

In [20]:
user_game.show()

+-----------------+------+
|         playerid|gameid|
+-----------------+------+
|76561198060698936|    60|
|76561198060698936|  1670|
|76561198060698936|  3830|
|76561198060698936|  1600|
|76561198060698936|  2900|
|76561198060698936|  2910|
|76561198060698936|  2920|
|76561198060698936|  4800|
|76561198060698936|  4000|
|76561198060698936|  6200|
|76561198060698936|  6210|
|76561198060698936|  6300|
|76561198060698936|  6900|
|76561198060698936|  6850|
|76561198060698936|  6860|
|76561198060698936|  6870|
|76561198060698936|  1610|
|76561198060698936|  1690|
|76561198060698936|  6980|
|76561198060698936|  7010|
+-----------------+------+
only showing top 20 rows



In [21]:
# Step 2: Keep only purchases by valid players
filtered_purchases_df = user_game.join(valid_players_df, on="playerid", how="inner")

In [22]:
filtered_purchases_df.show()

+-----------------+-------+
|         playerid| gameid|
+-----------------+-------+
|76561198389473114| 251570|
|76561198389473114|2923300|
|76561198389473114| 377160|
|76561198389473114| 252490|
|76561198389473114| 700580|
|76561198389473114| 225540|
|76561198389473114| 252950|
|76561198389473114| 427730|
|76561198389473114| 107410|
|76561198389473114| 431960|
|76561198389473114| 391220|
|76561198389473114| 265930|
|76561198389473114| 433340|
|76561198389473114| 271590|
|76561198389473114| 433910|
|76561198389473114| 438100|
|76561198389473114| 274170|
|76561198389473114| 441870|
|76561198389473114| 274190|
|76561198389473114| 444090|
+-----------------+-------+
only showing top 20 rows



In [30]:
# Step 3: Left join with achievement_counts
joined_df = filtered_purchases_df.join(
    achievement_counts,
    on=["playerid", "gameid"],
    how="left"
)

# Step 4: Fill nulls with 0 and add 1 for purchase bonus
final_df = joined_df.withColumn(
    "achievement_count",
    coalesce(col("achievement_count"), lit(0)) + lit(1)
)

In [24]:
final_df.show()

# check null value
# final_df.filter(final_df.gameid.isNull()).show()

+-----------------+-------+-----------------+
|         playerid| gameid|achievement_count|
+-----------------+-------+-----------------+
|76561198389473114| 107410|              105|
|76561198389473114| 225540|               67|
|76561198389473114| 252490|               55|
|76561198389473114| 265930|              128|
|76561198389473114| 271590|               78|
|76561198389473114| 431960|                6|
|76561198389473114| 596350|                1|
|76561198888702308|1029550|                5|
|76561198888702308| 105600|               70|
|76561198888702308| 107410|                8|
|76561198888702308|1209040|                1|
|76561198888702308|1293830|               55|
|76561198888702308|1407200|                1|
|76561198888702308| 231430|                1|
|76561198888702308| 236390|               27|
|76561198888702308| 244210|                4|
|76561198888702308|  43110|                1|
|76561198888702308|   4500|                1|
|76561198888702308| 582660|       

In [31]:
from pyspark.sql.functions import max

# Get max achievement count per game
max_achieve_per_game = final_df.groupBy("gameid") \
    .agg(max("achievement_count").alias("max_achievement"))

In [39]:
max_achieve_per_game.show()

# show max_achievement for gameid = 265930
max_achieve_per_game.filter(max_achieve_per_game.gameid == 265930).show()

+-------+---------------+
| gameid|max_achievement|
+-------+---------------+
| 332790|             16|
| 342580|             23|
| 530320|             76|
| 542400|             11|
| 652390|              1|
| 708630|              1|
| 738760|              1|
| 740410|             11|
| 825000|             10|
| 847600|              1|
| 108800|              1|
| 274560|             20|
|1144200|             24|
| 396750|             34|
|1303990|              4|
|1507190|              1|
| 232050|             57|
| 247020|             53|
| 282680|              1|
| 318220|              1|
+-------+---------------+
only showing top 20 rows

+------+---------------+
|gameid|max_achievement|
+------+---------------+
|265930|            128|
+------+---------------+



In [33]:
# Join back to the original DataFrame
joined_df = final_df.join(max_achieve_per_game, on="gameid", how="left")

# Step 3: Compute normalized implicit rating
final_df = joined_df.withColumn(
    "achievement_count",
    col("achievement_count") / col("max_achievement")
)

# Optional: Check the result
final_df.select("playerid", "gameid", "achievement_count").show()

+-----------------+-------+--------------------+
|         playerid| gameid|   achievement_count|
+-----------------+-------+--------------------+
|76561198888702308|1029550| 0.23809523809523808|
|76561198888702308| 244210|0.005633802816901409|
|76561198389473114| 596350|                 1.0|
|76561198389473114| 271590|                 1.0|
|76561198888702308|1407200|0.017241379310344827|
|76561198888702308| 231430|0.002217294900221...|
|76561198389473114| 265930|                 1.0|
|76561198389473114| 225540|                 1.0|
|76561198389473114| 431960|  0.3333333333333333|
|76561198888702308|1293830|  0.3089887640449438|
|76561198888702308|  43110|                 1.0|
|76561198389473114| 107410|  0.8467741935483871|
|76561198888702308| 107410| 0.06451612903225806|
|76561198888702308|1209040|                 1.0|
|76561198888702308| 582660|                 1.0|
|76561198888702308| 105600|   0.603448275862069|
|76561198888702308|   4500|                 1.0|
|76561198888702308| 

In [35]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer

In [36]:
player_indexer = StringIndexer(inputCol="playerid", outputCol="player_index", handleInvalid="skip")
game_indexer = StringIndexer(inputCol="gameid", outputCol="game_index", handleInvalid="skip")

# keep the mapping table for lookup later
player_id_mapping = player_indexer.fit(final_df)
game_id_mapping = game_indexer.fit(final_df)

# Fit and transform
indexed_df = player_id_mapping.transform(final_df)
indexed_df = game_id_mapping.transform(indexed_df)

In [51]:
indexed_df.show()

+-------+-----------------+--------------------+---------------+------------+----------+
| gameid|         playerid|   achievement_count|max_achievement|player_index|game_index|
+-------+-----------------+--------------------+---------------+------------+----------+
| 332790|76561197960272169|              0.0625|             16|        66.0|    2896.0|
| 342580|76561197960272169|0.043478260869565216|             23|        66.0|    2621.0|
| 530320|76561197960272169|0.013157894736842105|             76|        66.0|    2344.0|
| 542400|76561197960272169| 0.09090909090909091|             11|        66.0|    5108.0|
| 652390|76561197960272169|                 1.0|              1|        66.0|   11943.0|
| 708630|76561197960272169|                 1.0|              1|        66.0|   15496.0|
| 738760|76561197960272169|                 1.0|              1|        66.0|   19006.0|
| 740410|76561197960272169| 0.09090909090909091|             11|        66.0|    8445.0|
| 825000|765611979602

In [37]:
# Prepare for ALS
als_input = indexed_df.select(
    col("player_index").cast("int").alias("playerid"),
    col("game_index").cast("int").alias("gameid"),
    col("achievement_count").cast("float")
)

# Split into training and test sets
train_data, test_data = als_input.randomSplit([0.8, 0.2], seed=42)

In [38]:
# Initialize ALS for implicit feedback
als = ALS(
    userCol="playerid",
    itemCol="gameid",
    ratingCol="achievement_count",
    implicitPrefs=True,  # Crucial for implicit feedback
    coldStartStrategy="drop",  # Avoids NaN predictions
    rank=10,
    maxIter=15,
    regParam=0.1
)

# Train the model
model = als.fit(train_data)

In [40]:
# Predict on test data
predictions = model.transform(test_data)

# Optional: Evaluate with RMSE (less meaningful for implicit, but good for validation)
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="achievement_count",
    predictionCol="prediction"
)
rmse = evaluator.evaluate(predictions)
print(f"Test RMSE: {rmse:.4f}")

Test RMSE: 0.5157


In [45]:
user_recs = model.recommendForAllUsers(50)

In [48]:
from typing import List

def recommend_games_for_player(playerid: str,
                                user_recs_df,
                                player_id_mapping,
                                game_id_mapping,
                                player_game_df,
                                games_df,
                                spark) -> List[str]:

    # Map playerid to player_index
    if playerid not in player_id_mapping.labels:
        print(f"Playerid {playerid} not in training data.")
        return []

    player_index = float(player_id_mapping.labels.index(playerid))

    # Filter recommendations for this player_index
    recs = user_recs_df.filter(col("playerid") == player_index)

    # Explode recommendations and extract game_index
    exploded = recs.withColumn("rec", explode("recommendations")) \
                   .select(col("rec.gameid").alias("game_index"))

    # Map game_index back to gameid using model labels
    game_labels = game_id_mapping.labels
    recommended_gameids = exploded.rdd.map(lambda row: game_labels[int(row.game_index)]).collect()

    # Get list of already purchased gameids for this player
    purchased = player_game_df.filter(col("playerid") == playerid) \
                              .select("gameid").rdd.map(lambda r: str(r.gameid)).collect()

    # Filter out purchased games
    filtered_recommendations = [gid for gid in recommended_gameids if gid not in purchased]

    if not filtered_recommendations:
        print("No new games to recommend.")
        return

    # Convert to DataFrame for join
    rec_df = spark.createDataFrame([(gid,) for gid in filtered_recommendations], ["gameid"])

    # Join with metadata
    enriched_df = rec_df.join(games_df, on="gameid", how="left")

    # return max 10 result
    enriched_df = enriched_df.limit(10)

    # remove gameid with null title field
    enriched_df = enriched_df.filter(col("title").isNotNull())

    return enriched_df

In [43]:
recommended_games = recommend_games_for_player(
    playerid="76561197963765138",
    user_recs_df=user_recs,
    player_id_mapping=player_id_mapping,
    game_id_mapping=game_id_mapping,
    player_game_df=user_game,
    games_df=games,
    spark=spark
)

# show with no truncate
recommended_games.show(truncate=False)

+------+---------------------------------------+-----------------------------------+-------------------------+--------------------------------+----------------------+------------+
|gameid|title                                  |developers                         |publishers               |genres                          |supported_languages   |release_date|
+------+---------------------------------------+-----------------------------------+-------------------------+--------------------------------+----------------------+------------+
|391070|Stellar 2D                             |['quad games']                     |['Back To Basics Gaming']|['Action', 'Casual', 'Indie']   |['English']           |2015-07-30  |
|438030|Dead6hot                               |['Chuck Productions']              |['New Reality Games']    |['Action', 'Casual']            |['English']           |2016-02-01  |
|444480|Broken Dreams                          |['Mihai Morosanu']                 |['New Reality Ga

In [50]:
recommended_games = recommend_games_for_player(
    playerid="76561197969960651",
    user_recs_df=user_recs,
    player_id_mapping=player_id_mapping,
    game_id_mapping=game_id_mapping,
    player_game_df=user_game,
    games_df=games,
    spark=spark
)

# show with no truncate
recommended_games.show(truncate=False)

+------+---------------------------+-----------------------------------------------------------------+-----------------------+------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+
|gameid|title                      |developers                                                       |publishers             |genres                              |supported_languages                                                                                                                                                                                                                                                                                  